In [9]:
#Data Preparation
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset, DatasetDict
import numpy as np
import evaluate

# 1. Load the dataset and select 500 data points
dataset = load_dataset("imdb")

# Select a subset of 500 samples
train_subset = dataset["train"].select(range(500))
test_subset = dataset["test"].select(range(500))

subset_dataset = DatasetDict({
    "train": train_subset,
    "test": test_subset,
})




# 2. Tokenizer and Model
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

# 3. Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

#tokenized_datasets = subset_dataset.map(tokenize_function, batched=True)
tokenized_datasets = subset_dataset.map(tokenize_function, batched=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./sentiment_analysis",
        # Reduce the batch size if you don't have enough memory
        
        per_device_eval_batch_size=1,
        num_train_epochs=1
    ),
    
    eval_dataset=tokenized_datasets["test"],
    
    compute_metrics=compute_metrics,
    
)
#evaluate performance
trainer.evaluate()


{'eval_loss': 0.42025619745254517,
 'eval_model_preparation_time': 0.0027,
 'eval_accuracy': 0.906,
 'eval_runtime': 165.8762,
 'eval_samples_per_second': 3.014,
 'eval_steps_per_second': 3.014}

In [11]:
# 4. LORA Configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

model_lora = get_peft_model(model, lora_config)
model_lora.print_trainable_parameters()

# 5. Define the metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 6. Training Arguments
training_args = TrainingArguments(
    output_dir="./SentimentAnalysis",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    logging_steps=10,
    warmup_steps=500,
    gradient_accumulation_steps=2,
)

# 7. Trainer
trainer = Trainer(
    model=model_lora,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

# 8. Train the model
lora_trainer.train()

# 9. Evaluate the model
evaluation_results = lora_trainer.evaluate()
print(evaluation_results)

#model.save_pretrained("./lora_gpt2_imdb_sentiment")
#tokenizer.save_pretrained("./lora_gpt2_imdb_sentiment")

C:\Users\uak8880\AppData\Local\anaconda3\Lib\site-packages\peft\tuners\lora\layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 296,448 || all params: 124,737,792 || trainable%: 0.2377


C:\Users\uak8880\AppData\Local\anaconda3\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.313800,0.300241,0.938000
2,0.039200,0.014682,0.998000


{'eval_loss': 0.014682277105748653, 'eval_accuracy': 0.998, 'eval_runtime': 216.5584, 'eval_samples_per_second': 2.309, 'eval_steps_per_second': 0.577, 'epoch': 2.96}


In [ ]:
# Saving the model
model_lora.save_pretrained("/tmp/gp2-lora")

In [ ]:
from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("/tmp/gp2-lora")

In [ ]:
original_performance = trainer.evaluate()
fine_tuned_performance = lora_trainer.evaluate()
print("Original Model:", original_performance)
print("Fine-Tuned Model:", fine_tuned_performance)